# Use small manually-labeled set as examples and try few-shot of GPT model.

In [1]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
import pandas as pd
from langchain import PromptTemplate, OpenAI, LLMChain

In [41]:
import re
from sklearn.metrics import accuracy_score
import time

In [3]:
from config import *
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
sample_label_path = 'data/labelled_sample.csv'

In [5]:
df_sample = pd.read_csv(sample_label_path, encoding="GBK")

In [6]:
df_sample.head()

CONTENT TOPIC
0  推进基础设施建设和区域协调发展。京津冀协同发展、长江经济带建设取得重要进展。新建铁路投产里程...    基建
1  七是加快建立多主体供应、多渠道保障、租购并举的住房制度。要发展住房租赁市场特别是长期租赁，保...   房地产
2  会议认为，坚持标准引领，建设制造强国，是结构性改革尤其是供给侧结构性改革的重要内容，有利于改...   制造业
3  会议指出，党的十八大以来，党中央部署推进科技评价体系改革，聚焦“四个面向”的科技成果评价导向...    科技
4  会议强调，完善促进消费体制机制，要顺应居民消费新趋势，从供需两端发力，积极培育重点消费领域细...  零售消费

In [7]:
df_sample.shape

(24, 2)

In [8]:
examples = [{'content':row['CONTENT'],'topic':row['TOPIC']} for index, row in df_sample.iterrows()]

In [9]:
examples[:3]

[{'content': '推进基础设施建设和区域协调发展。京津冀协同发展、长江经济带建设取得重要进展。新建铁路投产里程8427公里，高速铁路运营里程达1.6万公里，占世界的60%以上。高速公路通车里程达11.2万公里，水路、民航、管道建设进一步加强。农网改造稳步进行。宽带用户超过7.8亿户。经过多年努力，南水北调中线一期工程正式通水，惠及沿线亿万群众。',
  'topic': '基建'},
 {'content': '七是加快建立多主体供应、多渠道保障、租购并举的住房制度。要发展住房租赁市场特别是长期租赁，保护租赁利益相关方合法权益，支持专业化、机构化住房租赁企业发展。完善促进房地产市场平稳健康发展的长效机制，保持房地产市场调控政策连续性和稳定性，分清中央和地方事权，实行差别化调控。',
  'topic': '房地产'},
 {'content': '会议认为，坚持标准引领，建设制造强国，是结构性改革尤其是供给侧结构性改革的重要内容，有利于改善供给、扩大需求，促进产品产业迈向中高端。会议通过了《装备制造业标准化和质量提升规划》，要求对接《中国制造2025》，瞄准国际先进水平，实施工业基础和智能制造、绿色制造标准化和质量提升工程，加快关键技术标准研制，推动在机器人、先进轨道交通装备、农业机械、高性能医疗器械等重点领域标准化实现新突破，并适应创新进展和市场需求及时更新标准，力争到2020年使重点领域国际标准转化率从目前的70%以上提高到90%以上。要弘扬工匠精神，追求精益求精，生产更多有创意、品质优、受群众欢迎的产品，坚决淘汰不达标产品，提振消费者对“中国制造”的信心，支撑制造业提质增效、提升国际竞争力。',
  'topic': '制造业'}]

In [10]:
pre_fix_template="""你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。
话题列表是：{topic_list}。
你的回答的格式必须是：回答-话题
请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。""".format(topic_list = ','.join(topics_pdf))

In [11]:
print(pre_fix_template)

你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。
话题列表是：基建,房地产,制造业,科技,零售消费,绿色发展,医药健康,教育,交通运输,出口贸易,农业农村,文化旅游,就业,民生扶贫,能源,物资储备,宏观,区域发展,企业发展,新冠疫情,地方财政,税务,国际关系,金融。
你的回答的格式必须是：回答-话题
请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。


In [12]:
example_prompt = PromptTemplate(input_variables=["content", "topic"], template="文字内容： {content}\n回答-话题：{topic}")

In [37]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    prefix=pre_fix_template,
    suffix="文字内容： {content}", 
    input_variables=['content']
)

print(prompt.format(content=examples[11]['content']))

你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。
话题列表是：基建,房地产,制造业,科技,零售消费,绿色发展,医药健康,教育,交通运输,出口贸易,农业农村,文化旅游,就业,民生扶贫,能源,物资储备,宏观,区域发展,企业发展,新冠疫情,地方财政,税务,国际关系,金融。
你的回答的格式必须是：回答-话题
请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。

文字内容： 推进基础设施建设和区域协调发展。京津冀协同发展、长江经济带建设取得重要进展。新建铁路投产里程8427公里，高速铁路运营里程达1.6万公里，占世界的60%以上。高速公路通车里程达11.2万公里，水路、民航、管道建设进一步加强。农网改造稳步进行。宽带用户超过7.8亿户。经过多年努力，南水北调中线一期工程正式通水，惠及沿线亿万群众。
回答-话题：基建

文字内容： 七是加快建立多主体供应、多渠道保障、租购并举的住房制度。要发展住房租赁市场特别是长期租赁，保护租赁利益相关方合法权益，支持专业化、机构化住房租赁企业发展。完善促进房地产市场平稳健康发展的长效机制，保持房地产市场调控政策连续性和稳定性，分清中央和地方事权，实行差别化调控。
回答-话题：房地产

文字内容： 会议认为，坚持标准引领，建设制造强国，是结构性改革尤其是供给侧结构性改革的重要内容，有利于改善供给、扩大需求，促进产品产业迈向中高端。会议通过了《装备制造业标准化和质量提升规划》，要求对接《中国制造2025》，瞄准国际先进水平，实施工业基础和智能制造、绿色制造标准化和质量提升工程，加快关键技术标准研制，推动在机器人、先进轨道交通装备、农业机械、高性能医疗器械等重点领域标准化实现新突破，并适应创新进展和市场需求及时更新标准，力争到2020年使重点领域国际标准转化率从目前的70%以上提高到90%以上。要弘扬工匠精神，追求精益求精，生产更多有创意、品质优、受群众欢迎的产品，坚决淘汰不达标产品，提振消费者对“中国制造”的信心，支撑制造业提质增效、提升国际竞争力。
回答-话题：制造业

文字内容： 会议指出，党的十八大以来，党中央部署推进科技评价体系改革，聚焦“四个面向”的科技成果评价导向逐步确立，激励创新的环境正在形成，带动我国科技创新能力明显提升

In [17]:
llm = OpenAI(temperature = 0, openai_api_key = OPENAI_API_KEY, model_name = 'gpt-4')

D:\Users\shaodongxu\anaconda3\lib\site-packages\langchain\llms\openai.py:189: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
D:\Users\shaodongxu\anaconda3\lib\site-packages\langchain\llms\openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [38]:
llm_chain = LLMChain(
    llm = llm,
    prompt = prompt
)

In [45]:
df_sample_test = pd.read_csv('data/labelled_test.csv', encoding="GBK")

In [46]:
df_sample_test.head()

CONTENT TOPIC
0  重大水利工程是“两新一重”建设的重要内容。会议围绕防洪减灾、水资源优化配置、水生态保护修复等...    基建
1  会议强调，要有效管控重点风险，守住不发生系统性风险底线。要坚持房子是用来住的、不是用来炒的定...   房地产
2  国务院总理李克强8月24日主持召开国务院常务会议，部署促进消费品标准和质量提升，增加“中国制...   制造业
3  实施创新驱动发展战略。着力打通科技成果转化通道，扩大中关村国家自主创新示范区试点政策实施范围...    科技
4  加快培育消费增长点。鼓励大众消费，控制“三公”消费。促进养老家政健康消费，壮大信息消费，提升...  零售消费

In [47]:
df_sample_test.shape

(24, 2)

In [48]:
df_sample_test['CONTENT'][10]

'会议指出，探索建立涉农资金统筹整合长效机制，是推进农业供给侧结构性改革的重要途径。要加强财政支农政策顶层设计，优化财政支农投入供给，推进行业内资金整合与行业间资金统筹相互衔接配合，理顺涉农资金管理体系，创新涉农资金使用管理机制，改革和完善农村投融资体制，切实提高国家支农政策效果和支农资金使用效益。'

In [49]:
pred = []
for index, row in df_sample_test.iterrows():
    pred.append(llm_chain.predict(content=row['CONTENT']).split('话题：')[-1])
    time.sleep(65)
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [51]:
accuracy_score(df_sample_test['TOPIC'],pred)

0.7916666666666666

In [62]:
0.7916666666666666 * 24

19.0

In [53]:
df_sample_test['TOPIC_FEW'] = pred

In [54]:
df_sample_test['TOPIC_FEW'].value_counts()

金融      3
企业发展    2
民生扶贫    2
基建      1
军民融合    1
区域发展    1
宏观      1
物资储备    1
能源      1
文化旅游    1
农业农村    1
出口贸易    1
交通运输    1
教育      1
医药健康    1
绿色发展    1
零售消费    1
科技      1
制造业     1
国际关系    1
Name: TOPIC_FEW, dtype: int64

## Zero-shot

In [55]:
zero_prompt = pre_fix_template + "\n文字内容：{content}"

In [56]:
print(zero_prompt)

你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。
话题列表是：基建,房地产,制造业,科技,零售消费,绿色发展,医药健康,教育,交通运输,出口贸易,农业农村,文化旅游,就业,民生扶贫,能源,物资储备,宏观,区域发展,企业发展,新冠疫情,地方财政,税务,国际关系,金融。
你的回答的格式必须是：回答-话题
请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。
文字内容：{content}


In [57]:
llm_chain_zero = LLMChain(
    llm = llm,
    prompt = PromptTemplate.from_template(zero_prompt)
)

In [58]:
llm_chain_zero.predict(content = df_sample_test['CONTENT'][10])

'回答-农业农村'

In [59]:
df_sample_test['PRED_ZERO'] = df_sample_test.apply({lambda row: llm_chain_zero.predict(content=row['CONTENT']).split('-')[-1]},
                                             axis = 1)

In [60]:
df_sample_test['PRED_ZERO'].value_counts()

民生扶贫    3
交通运输    2
基建      1
文化旅游    1
国际关系    1
税务      1
地方财政    1
企业发展    1
区域发展    1
宏观      1
能源      1
农业农村    1
房地产     1
出口贸易    1
教育      1
医药健康    1
绿色发展    1
零售消费    1
科技      1
制造业     1
金融      1
Name: PRED_ZERO, dtype: int64

In [61]:
accuracy_score(df_sample_test['TOPIC'],df_sample_test['PRED_ZERO'])

0.875

In [63]:
0.875 *24

21.0